In [2]:
f = open("bloom-syndrome-protein.txt", "r")
lines = f.readlines()
genome = ''.join(''.join(lines).split())
genome

'gaagatgctcaggaaagtgactctctgaaaactcatttggaagatgaaagagataatagcgaaaagaagaagaatttggaagaagctgaattacattcaactgagaaagttccatgtattgaatttgatgatgatgattatgatacggattttgttccaccttctccagaagaaattatttctgcttcttcttcctcttcaaaatgccttagtacgttaaaggaccttgacacctctgacagaaaagaggatgttcttagcacatcaaaagatcttttgtcaaaacctgagaaaatgagtatgcaggagctgaatccagaaaccagcacagactgtgacgctagacagataagtttacagcagcagcttattcatgtgatggagcacatctgtaaattaattgatactattcctgatgataaactgaaacctttggattgtgggaacgaactgcttcagcagcggaacataagaaggaaacttctaacggaagtagattttaataaaagtgatgccagtcttcttggctcattgtggagatacaggcctgattcacttgatggccctatggagggtgattcctgccctacagggaattctatgaaggagttaaatttttcacaccttccctcaaattctgtttctcctggggactgtttactgactaccaccctaggaaagacaggattctctgccaccaggaagaatctttttgaaaggcctttattcaatacccatttacagaagtcctttgtaagtagcaactgggctgaaacaccaagactaggaaaaaaaaatgaaagctcttatttcccaggaaatgttctcacaagcactgctgtgaaagatcagaataaacatactgcttcaataaatgacttagaaagagaaacccaaccttcctatgatattgataattttgacatagatgactttgatgatgatgatgactgggaagacataatgcataatttagcagccagcaaatcttccacagct

In [3]:
# Basic Run length encoding 
MAX_RUN = 4 #not being used currently
genome_size = len(genome)

def run_length_coding(sequence):
    compressed = ""
    size = len(sequence)
    i = 0
    while i < size: 
        run_length = 1
        k = i+1
        while k < size and sequence[i]==sequence[k]:
            run_length += 1
            k += 1
    
        compressed += sequence[i] + str(run_length)*( 0 if run_length==1 else 1)
        i = k   
    return compressed

result = run_length_coding(genome)
len(result)

3139

In [4]:
# Gzip compression
import gzip
compressed_gzip = gzip.compress(bytes(genome, 'utf-8'))
print (len(compressed_gzip))

1120


In [5]:
# Bzip2 compression
import bz2
compressed_bzip = bz2.compress(bytes(genome, 'utf-8'))
print (len(compressed_bzip))

1020


In [9]:
# Split encoding
a_rle = []
g_rle = []
c_rle = []
t_rle = []
for i in range(genome_size):
    if genome[i] == 'a':
        a_rle.append('1')
    else:
        a_rle.append('0')

    if genome[i] == 'g':
        g_rle.append('1')
    else:
        g_rle.append('0')
    if genome[i] == 'c':
        c_rle.append('1')
    else:
        c_rle.append('0')
    if genome[i] == 't':
        t_rle.append('1')
    else:
        t_rle.append('0')

def split_encoding(sequence):
    sequence = ['0'] + sequence #start every sequence with 0
    compressed = ""
    size = len(sequence)
    i = 0
    while i < size: 
        run_length = 1
        k = i+1
        while k < size and sequence[i]==sequence[k]:
            run_length += 1
            k += 1
        compressed += sequence[i] + str(run_length)
        i = k
    new_compress = ''
    print (compressed[164:189], compressed[189])
    for i in range(len(compressed)):
        if i%2==1:
            new_compress += compressed[i] + f"-{i} : "
    return new_compress


print (split_encoding(a_rle))
# print (split_encoding(g_rle))


0211022140611021104130211 0
2-1 : 2-3 : 1-5 : 1-7 : 5-9 : 1-11 : 2-13 : 3-15 : 3-17 : 1-19 : 7-21 : 4-23 : 3-25 : 1-27 : 5-29 : 2-31 : 1-33 : 1-35 : 2-37 : 3-39 : 1-41 : 1-43 : 1-45 : 1-47 : 1-49 : 2-51 : 1-53 : 1-55 : 3-57 : 4-59 : 1-61 : 2-63 : 1-65 : 2-67 : 1-69 : 2-71 : 5-73 : 2-75 : 1-77 : 2-79 : 4-81 : 2-83 : 2-85 : 1-87 : 1-89 : 1-91 : 3-93 : 2-95 : 3-97 : 1-99 : 1-101 : 3-103 : 5-105 : 1-107 : 3-109 : 1-111 : 3-113 : 2-115 : 4-117 : 1-119 : 2-121 : 1-123 : 2-125 : 1-127 : 2-129 : 1-131 : 2-133 : 1-135 : 2-137 : 1-139 : 1-141 : 1-143 : 3-145 : 1-147 : 9-149 : 1-151 : 8-153 : 1-155 : 1-157 : 2-159 : 1-161 : 3-163 : 2-165 : 1-167 : 2-169 : 1-171 : 0-173 : 1-175 : 0-177 : 1-179 : 0-181 : 1-183 : 0-185 : 1-187 : 0-189 : 1-191 : 0-193 : 1-195 : 0-197 : 1-199 : 0-201 : 1-203 : 0-205 : 1-207 : 0-209 : 1-211 : 0-213 : 1-215 : 0-217 : 1-219 : 0-221 : 1-223 : 0-225 : 1-227 : 0-229 : 1-231 : 0-233 : 1-235 : 0-237 : 1-239 : 0-241 : 1-243 : 0-245 : 1-247 : 0-249 : 1-251 : 0-253 : 1-255 : 0-2